<a href="https://colab.research.google.com/github/arjunverma2004/ANN-CustomerChurn-classification/blob/main/notebooks/hyperparametertuningann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN)
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [3]:
!pip install -q -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.8 MB/s eta 0:00:00


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle
import keras_tuner as kt

In [5]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [10]:
from tensorflow.keras.layers import Dropout
def build_model(hp):
  model = Sequential()

  counter = 0
  #We used in if condn to create input layer


  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    if counter == 0:
      model.add(Dense(
          hp.Int('units' + str(i),min_value=8,max_value=128,step=8),
          activation= hp.Choice('activation'+ str(i), values=['relu','tanh','sigmoid']),
          input_shape=(X_train.shape[1],) ))
      model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.4,0.5,0.6,0.7,0.8,0.9])))

    else:
      model.add(Dense(
          hp.Int('units' + str(i),min_value=8,max_value=128,step=8),
          activation= hp.Choice('activation'+ str(i), values=['relu','tanh','sigmoid'])))
      model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter += 1

  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',['adam','sgd','rmsprop','adadelta']),
                loss='binary_crossentropy', metrics=['accuracy'])

  return model



In [13]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=10)

In [14]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Trial 10 Complete [00h 00m 27s]
val_accuracy: 0.8034999966621399

Best val_accuracy So Far: 0.8585000038146973
Total elapsed time: 00h 03m 07s


In [15]:
tuner.get_best_hyperparameters()[0].values


{'num_layers': 4,
 'units0': 96,
 'activation0': 'tanh',
 'dropout0': 0.1,
 'optimizer': 'rmsprop',
 'units1': 72,
 'activation1': 'relu',
 'dropout1': 0.6,
 'units2': 16,
 'activation2': 'tanh',
 'dropout2': 0.6,
 'units3': 80,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 96,
 'activation4': 'tanh',
 'dropout4': 0.2,
 'units5': 16,
 'activation5': 'sigmoid',
 'dropout5': 0.7,
 'units6': 104,
 'activation6': 'relu',
 'dropout6': 0.9}

In [16]:
model = tuner.get_best_models(num_models=1)[0]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 96)             │         1,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 72)             │         6,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 72)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 80)             │         1,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,841 (42.35 KB)

 Trainable params: 10,841 (42.35 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), intial_epochs=10, callbacks=[EarlyStopping(monitor='val_loss', patience=10)])

In [ ]:

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

###Without Dropouts

In [18]:

def build_model1(hp):
  model = Sequential()

  counter = 0
  #We used in if condn to create input layer


  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    if counter == 0:
      model.add(Dense(
          hp.Int('units' + str(i),min_value=8,max_value=128,step=8),
          activation= hp.Choice('activation'+ str(i), values=['relu','tanh','sigmoid']),
          input_shape=(X_train.shape[1],) ))


    else:
      model.add(Dense(
          hp.Int('units' + str(i),min_value=8,max_value=128,step=8),
          activation= hp.Choice('activation'+ str(i), values=['relu','tanh','sigmoid'])))

    counter += 1

  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',['adam','sgd','rmsprop','adadelta']),
                loss='binary_crossentropy', metrics=['accuracy'])

  return model


In [20]:
tuner1 = kt.RandomSearch(build_model1,
                        objective='val_accuracy',
                        max_trials=10,
                         directory='nodrop')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
tuner1.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Trial 10 Complete [00h 00m 15s]
val_accuracy: 0.8034999966621399

Best val_accuracy So Far: 0.8604999780654907
Total elapsed time: 00h 02m 47s


In [23]:
model1 = tuner1.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [25]:
model1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 120)            │         3,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           121 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,497 (17.57 KB)

 Trainable params: 4,497 (17.57 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
model1.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), initial_epoch=10, callbacks=[EarlyStopping(monitor='val_loss', patience=10)])

Epoch 11/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8590 - loss: 0.3487 - val_accuracy: 0.8605 - val_loss: 0.3442
Epoch 12/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8645 - loss: 0.3377 - val_accuracy: 0.8555 - val_loss: 0.3544
Epoch 13/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8573 - loss: 0.3450 - val_accuracy: 0.8605 - val_loss: 0.3456
Epoch 14/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8627 - loss: 0.3283 - val_accuracy: 0.8645 - val_loss: 0.3456
Epoch 15/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8626 - loss: 0.3298 - val_accuracy: 0.8575 - val_loss: 0.3461
Epoch 16/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8606 - loss: 0.3359 - val_accuracy: 0.8575 - val_loss: 0.3481
Epoch 17/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8626 - loss: 0.3209 - val_accuracy: 0.8660 - val_loss: 0.3426
Epoch 18/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8648 - loss: 0.3253 - 

In [28]:
loss, accuracy = model1.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8638 - loss: 0.3396
Test Loss: 0.3485
Test Accuracy: 0.8620


In [30]:
model1.save('model1.h5')